In [1]:
import pandas as pd

import numpy as np

In [2]:
comment = pd.read_json('UBS Weibo Data//weibo//Status_Data9_Comment_Data_with_city_code.json', encoding= 'UTF-8')
city = pd.read_json('UBS Weibo Data//weibo//weibo-china-province-city.json', encoding= 'UTF-8')

In [3]:
city['ProvinceID_CityID'] = city['province'].apply(str)+"_"+city['city'].apply(str)

In [4]:
comment['ProvinceID_CityID'] = comment['u_province'].apply(str)+"_"+comment['u_city'].apply(str)

In [5]:
comment = comment[~(comment["ProvinceID_CityID"]== '_')]

In [6]:
city.head()

,city,city_en,city_sc,city_tc,province,province_en,province_sc,province_tc,ref_id,ProvinceID_CityID
0,1,Dongcheng,东城,東城,11,Beijing,北京,北京,1011001,11_1
1,2,Xicheng,西城,西城,11,Beijing,北京,北京,1011002,11_2
2,3,Chongwen,崇文,崇文,11,Beijing,北京,北京,1011003,11_3
3,4,Xuanwu,宣武,宣武,11,Beijing,北京,北京,1011004,11_4
4,5,Chaoyang,朝阳,朝陽,11,Beijing,北京,北京,1011005,11_5


In [7]:
comment.head()

,cid,u_city,u_name,u_province,uid,ProvinceID_CityID
0,G6RevbiBE,2,了不起的查尔斯,11,5048555731,11_2
1,G6RevbiBE,5,Pokerface1001,11,3211419333,11_5
2,G6RevbiBE,2,了不起的查尔斯,11,5048555731,11_2
3,G6QJ1aa2Z,3,飞马1515,44,5583941399,44_3
4,G9Ob5sz8r,1000,原来是酱紫的呀,400,5364706506,400_1000


In [8]:
task = pd.read_csv("task.txt",header=None, sep="\t")

In [9]:
test = pd.read_csv("prediction.txt",header=None, sep="\t")

In [10]:
task = task.append(test,ignore_index=True)

In [11]:
task = task.drop_duplicates()

In [12]:
task.head()

,0,1,2,3
0,G5jCq7d7M,8,16.0,16.0
1,G83Zh7dWB,10,21.0,8.0
2,G2iM478Ip,8,36.0,114.0
3,GaYEQ8hI3,9,11.0,2.0
4,G2iCowAOT,0,19.0,14.0


In [13]:
task.columns = ["cid", "label",'likes_count','retweets_count']

In [14]:
comment = pd.merge(comment,task,on="cid",how="inner")

In [15]:
comment.head()

,cid,u_city,u_name,u_province,uid,ProvinceID_CityID,label,likes_count,retweets_count
0,G6RevbiBE,2,了不起的查尔斯,11,5048555731,11_2,9,26.0,5.0
1,G6RevbiBE,5,Pokerface1001,11,3211419333,11_5,9,26.0,5.0
2,G6RevbiBE,2,了不起的查尔斯,11,5048555731,11_2,9,26.0,5.0
3,G6RevbiBE,3,杜浔阿标,44,3221877842,44_3,9,26.0,5.0
4,G6RevbiBE,3,飞马1515,44,5583941399,44_3,9,26.0,5.0


In [16]:
comment["label"] = comment["label"].apply(str)

In [17]:
comment[comment["ProvinceID_CityID"]== '_']

,cid,u_city,u_name,u_province,uid,ProvinceID_CityID,label,likes_count,retweets_count


In [18]:
likes = pd.crosstab(comment.ProvinceID_CityID, comment.label, values = comment.likes_count, aggfunc=[np.mean])
likes = likes.fillna(likes.mean())

In [19]:
comments = pd.crosstab(comment.ProvinceID_CityID, comment.label)

In [20]:
retweets = pd.crosstab(comment.ProvinceID_CityID, comment.label, values = comment.retweets_count, aggfunc=[np.mean])
retweets = retweets.fillna(retweets.mean())

In [21]:
df1 = (likes*0.4+comments*0.25+retweets*0.35).rank(axis=1, ascending=False)

In [22]:
df1.head(10)

mean                                                     \
label                 0    1    10    11    12    4     5     6     7    8   
ProvinceID_CityID                                                            
100_1000            7.0  6.0   3.0   9.0   1.0  5.0  10.0   8.0  11.0  4.0   
11_1                7.0  9.0   3.0   5.0   2.0  6.0   8.0  10.0  11.0  4.0   
11_1000             8.0  4.0   5.0  10.0   2.0  6.0   9.0   7.0  11.0  3.0   
11_11              11.0  3.0   6.0   8.0   1.0  5.0   9.0   7.0  10.0  4.0   
11_12               6.0  4.0   8.0   7.0  11.0  5.0   9.0   3.0  10.0  2.0   
11_13               3.0  4.0   6.0   9.0   2.0  5.0  10.0   8.0  11.0  7.0   
11_14               5.0  3.0   7.0   6.0  11.0  4.0   8.0  10.0   9.0  1.0   
11_15               9.0  4.0   8.0   7.0   1.0  5.0  10.0   6.0  11.0  3.0   
11_16               6.0  2.0   5.0   8.0   1.0  4.0   9.0   7.0  10.0  3.0   
11_17               6.0  3.0  11.0   8.0   1.0  5.0   9.0   7.0  10.0  4.0   

                         
label                 9  
ProvinceID_CityID        
100_1000            2.0  
11_1                1.0  
11_1000             1.0  
11_11               2.0  
11_12               1.0  
11_13               1.0  
11_14               2.0  
11_15               2.0  
11_16              11.0  
11_17               2.0

In [23]:
d = {"Rank_1":[],"Rank_2":[],"Rank_3":[],"Rank_4":[],"Rank_5":[],"Rank_6":[],"Rank_7":[],"Rank_8":[],"Rank_9":[],"Rank_10":[]}
for i in df1.index:
    for j in range(1,11):
        try:
            x = df1.ix[i][(df1.ix[i]==float(j))].index[0][1]
            d["Rank_{}".format(j)].append(x)
        except:
            d["Rank_{}".format(j)].append(np.nan)
rank_w_city = pd.DataFrame(d, index=df1.index)

C:\Users\edwin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [24]:
rank_w_city

,Rank_1,Rank_10,Rank_2,Rank_3,Rank_4,Rank_5,Rank_6,Rank_7,Rank_8,Rank_9
ProvinceID_CityID,,,,,,,,,,
100_1000,12,5,9,10,8,4,1,0,6,11
11_1,9,6,12,10,8,11,4,0,5,1
11_1000,9,11,12,8,1,10,4,6,0,5
11_11,12,7,9,1,8,4,10,6,11,5
11_12,9,7,8,6,1,4,0,11,10,5
11_13,9,5,12,0,1,4,10,8,6,11
11_14,8,6,9,1,4,0,11,10,5,7
11_15,12,5,9,8,1,4,6,11,10,0
11_16,12,7,1,8,4,10,0,6,11,5


In [25]:
likes = pd.crosstab(comment.label,comment.ProvinceID_CityID, values = comment.likes_count, aggfunc=[np.mean])
likes = likes.fillna(likes.mean())

In [26]:
comments = pd.crosstab(comment.label, comment.ProvinceID_CityID)

In [27]:
retweets = pd.crosstab(comment.label, comment.ProvinceID_CityID, values = comment.retweets_count, aggfunc=[np.mean])
retweets = retweets.fillna(retweets.mean())

In [28]:
df2 = (likes*0.4+comments*0.25+retweets*0.35).rank(axis=1, ascending=False)

In [29]:
df2.head()

mean                                                    \
ProvinceID_CityID 100_1000   11_1 11_1000  11_11  11_12  11_13  11_14  11_15   
label                                                                          
0                    203.0  271.0   300.0  454.0  104.0  110.5  175.0  339.0   
1                    282.0  397.0   226.0  402.0  169.0   89.0  263.0  246.0   
10                    21.0   47.0   102.0  254.0  306.0  135.0  352.0  309.5   
11                   239.0  238.0   379.0  376.0  120.0   54.0  200.0  140.0   
12                    59.0  136.0   120.0  372.0  425.0  123.0  441.0  182.0   

                                 ...                                       \
ProvinceID_CityID  11_16  11_17  ...    81_13   81_2   81_4   81_9   82_1   
label                            ...                                        
0                  482.0  443.5  ...    252.0  209.0   59.0  459.0  143.0   
1                  482.0  454.5  ...    315.0  396.0  104.0  468.0  210.0   
10                 481.0  437.5  ...    192.0  342.0  379.0  460.0  351.0   
11                 480.0  435.5  ...    253.0  409.0   67.0  454.5  154.0   
12                 479.0  435.5  ...    280.5  289.0  438.0  452.0  192.0   

                                                       
ProvinceID_CityID 82_1000   82_2   82_3   82_4   82_6  
label                                                  
0                   420.5  126.0  193.0  110.5  384.5  
1                   428.0  186.0  281.0  171.0  408.5  
10                  474.0  453.0  133.0   42.0  366.5  
11                  162.5  134.0  217.0  122.0  385.5  
12                  354.0  177.0  254.0  170.0  384.5  

[5 rows x 483 columns]

In [30]:
d = {"Rank_1":[],"Rank_2":[],"Rank_3":[],"Rank_4":[],"Rank_5":[],"Rank_6":[],"Rank_7":[],"Rank_8":[],"Rank_9":[],"Rank_10":[]}
for i in df2.index:
    for j in range(1,11):
        try:
            x = df2.ix[i][(df2.ix[i]==float(j))].index[0][1]
            d["Rank_{}".format(j)].append(x)
        except:
            d["Rank_{}".format(j)].append(np.nan)
rank_a_city = pd.DataFrame(d, index=df2.index)

C:\Users\edwin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [31]:
rank_a_city

,Rank_1,Rank_10,Rank_2,Rank_3,Rank_4,Rank_5,Rank_6,Rank_7,Rank_8,Rank_9
label,,,,,,,,,,
0,42_30,15_5,400_10,21_14,45_4,12_7,41_17,50_2,53_32,14_4
1,42_30,15_5,400_10,21_14,45_4,12_7,41_17,50_2,53_32,14_4
10,42_30,15_2,400_10,21_14,52_23,23_9,45_12,14_6,50_41,21_9
11,42_30,15_5,400_10,21_14,45_4,12_7,41_17,50_2,53_32,14_4
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42_30,400_19,400_10,21_14,12_7,41_17,50_2,53_32,15_5,45_2
5,42_30,15_5,400_10,21_14,45_4,12_7,41_17,50_2,53_32,14_4
6,42_30,15_5,400_10,21_14,45_4,12_7,41_17,50_2,53_32,14_4
7,42_30,15_5,400_10,21_14,45_4,12_7,41_17,50_2,53_32,14_4


In [33]:
rank_w_city.to_csv('rank_w_city.csv')

In [34]:
rank_a_city.to_csv('rank_a_city.csv')